# Lab: Visualization and Imputation (AAVAIL, US & Singapore)

This notebook follows a guided lab structure to analyze the AAVAIL dataset. We will focus on:

1.  Loading and inspecting the data.
2.  Identifying and visualizing missing data.
3.  Imputing missing values using appropriate techniques.
4.  Visualizing business insights for the US and Singapore markets.
5.  Summarizing takeaways for stakeholders.

**Libraries Used:**

* `pandas`
* `numpy`
* `matplotlib`
* `seaborn`


# --- 1. Setup and Data Loading ---


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set a consistent style for all plots
sns.set_theme(style="whitegrid")

# Configure paths
BASE_DIR = Path("..")
RAW_DATA_PATH = BASE_DIR / "data" / "raw" / "aavail_customer_activity.csv"
PROCESSED_DATA_PATH = BASE_DIR / "data" / "processed" / "aavail_data_imputed.csv"
REPORT_PATH = BASE_DIR / "reports" / "aavail_market_insights.html"
ASSETS_DIR = BASE_DIR / "reports" / "assets"
ASSETS_DIR.mkdir(parents=True, exist_ok=True)


## Step 1: Load and Inspect Data

Load the dataset using Pandas and inspect its structure and summary statistics.


In [ ]:
# Load the dataset
file_name = RAW_DATA_PATH

try:
    df = pd.read_csv(file_name)
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print(f"Error: The file '{file_name}' was not found.")
    print("Please upload the file to your Colab session and try again.")
    # Create a dummy dataframe to allow subsequent cells to run without error
    df = pd.DataFrame()

if not df.empty:
    # Display the first 5 rows
    print("\n--- Head ---")
    print(df.head())
    
    # Display data structure and types
    print("\n--- Info ---")
    df.info()
    
    # Display summary statistics for numerical columns
    print("\n--- Describe (Numerical) ---")
    print(df.describe())
    
    # Display summary for categorical columns
    print("\n--- Describe (Categorical) ---")
    print(df.describe(include=['object', 'bool']))


In [ ]:
## Step 2: Identify and Visualize Missing Data

We will create a heatmap to quickly identify patterns in missing data and then summarize the exact percentages.


In [ ]:
if not df.empty:
    # Create a missingness heatmap
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
    plt.title('Missing Data Heatmap')
    plt.tight_layout()
    missing_heatmap_path = ASSETS_DIR / "missingness_heatmap.png"
    plt.savefig(missing_heatmap_path, dpi=150, bbox_inches="tight")
    plt.show()
    
    # Summarize missing data per column
    missing_data = df.isnull().sum()
    missing_percentage = (missing_data / len(df)) * 100
    missing_summary = pd.DataFrame({
        'Missing Count': missing_data,
        'Missing Percentage': missing_percentage
    })
    print("\n--- Missing Data Summary ---")
    print(missing_summary.sort_values(by='Missing Count', ascending=False))
    
    overall_percentage = (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
    print(f"\nOverall percentage of missing data: {overall_percentage:.2f}%")


**Missing Data Analysis:**

The heatmap and summary table show three columns with missing data:

1.  `subscriber_type` (Categorical)
2.  `num_streams` (Numeric)
3.  `age` (Numeric)

`subscriber_type` is missing a significant number of values, which is problematic as it's a key business metric. `num_streams` and `age` are missing fewer values.

## Step 3: Impute Missing Data

We will impute the missing values.

* **`age` (Numeric):** We will use the **median** value. The median is more robust to outliers than the mean, and a quick look at `df.describe()` (min 15, max 75) suggests a standard age range, but median is a safer choice.
* **`num_streams` (Numeric):** We will also use the **median**.
* **`subscriber_type` (Categorical):** We will use the **mode** (the most frequent value). This is a standard approach for categorical data.

We will create a copy of the dataframe for imputation to show "before vs. after."


In [ ]:
if not df.empty:
    # Create a copy for imputation
    df_imputed = df.copy()
    
    # --- Imputation Strategy ---
    # 1. For 'age' and 'num_streams', use the median
    median_age = df_imputed['age'].median()
    median_streams = df_imputed['num_streams'].median()
    
    # 2. For 'subscriber_type', use the mode
    # .mode() returns a Series, so we take the first item [0]
    mode_subscriber_type = df_imputed['subscriber_type'].mode()[0]
    
    print(f"Imputing 'age' with median: {median_age}")
    print(f"Imputing 'num_streams' with median: {median_streams}")
    print(f"Imputing 'subscriber_type' with mode: '{mode_subscriber_type}'")
    
    # --- Apply Imputation ---
    df_imputed['age'].fillna(median_age, inplace=True)
    df_imputed['num_streams'].fillna(median_streams, inplace=True)
    df_imputed['subscriber_type'].fillna(mode_subscriber_type, inplace=True)
    
    # --- Verify Cleanup ---
    print("\n--- Missing Data Summary (After Imputation) ---")
    print(df_imputed.isnull().sum())
    
    # Save the imputed dataset
    df_imputed.to_csv(PROCESSED_DATA_PATH, index=False)
    print(f"\nImputed dataset saved to {PROCESSED_DATA_PATH}")


### Visualize Imputation: Before and After

Let's plot the distributions for `age` and `num_streams` to ensure our imputation didn't drastically skew the data.


In [ ]:
if not df.empty:
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle('Distribution Before vs. After Imputation', fontsize=16, y=1.02)
    
    # --- Age ---
    # Before
    sns.histplot(df['age'].dropna(), kde=True, ax=axes[0, 0], color='blue', label='Original')
    axes[0, 0].set_title('Age Distribution (Before)')
    
    # After
    sns.histplot(df_imputed['age'], kde=True, ax=axes[0, 1], color='green', label='Imputed')
    axes[0, 1].set_title('Age Distribution (After)')
    
    # --- Num Streams ---
    # Before
    sns.histplot(df['num_streams'].dropna(), kde=True, ax=axes[1, 0], color='blue', label='Original')
    axes[1, 0].set_title('Num Streams Distribution (Before)')
    
    # After
    sns.histplot(df_imputed['num_streams'], kde=True, ax=axes[1, 1], color='green', label='Imputed')
    axes[1, 1].set_title('Num Streams Distribution (After)')
    
    plt.tight_layout()
    plt.savefig(ASSETS_DIR / "imputation_comparison.png", dpi=150, bbox_inches="tight")
    plt.show()


In [ ]:
**Imputation Visualization Analysis:**

The "After" plots are very similar to the "Before" plots. The imputation (the green line) has not significantly changed the overall shape of the distributions. This is good! It means our choice of median was appropriate and didn't introduce a strong bias. The small number of missing values for these features also contributed to this minimal impact.

## Step 4: Clean Dataset

Our imputation step has handled all missing values. We will now proceed with the cleaned dataset, `df_imputed`.


In [ ]:
if not df.empty:
    print("Dataset is clean and ready for analysis.")
    print(f"Total rows: {len(df_imputed)}")


In [ ]:
## Step 5: Visualize Business Insights (US vs. Singapore)

Now we'll use the cleaned data to compare the two markets.


In [ ]:
if not df.empty:
    # Separate data by market
    df_us = df_imputed[df_imputed['country_name'] == 'united_states'].copy()
    df_sg = df_imputed[df_imputed['country_name'] == 'singapore'].copy()
    
    print(f"US Market Rows: {len(df_us)}")
    print(f"Singapore Market Rows: {len(df_sg)}")


### Insight 1: Subscriber Type Distribution by Market

How popular is each subscription plan in each country?


if not df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
    fig.suptitle('Subscriber Type Distribution by Market')
    
    # US Market
    sns.countplot(data=df_us, x='subscriber_type', ax=axes[0], palette='coolwarm',
                  order=df_imputed['subscriber_type'].value_counts().index)
    axes[0].set_title('United States')
    axes[0].set_xlabel('Subscriber Type')
    axes[0].set_ylabel('Count of Customers')
    
    # Singapore Market
    sns.countplot(data=df_sg, x='subscriber_type', ax=axes[1], palette='coolwarm',
                  order=df_imputed['subscriber_type'].value_counts().index)
    axes[1].set_title('Singapore')
    axes[1].set_xlabel('Subscriber Type')
    axes[1].set_ylabel('') # Remove y-label for clarity
    
    plt.tight_layout()
    plt.savefig(ASSETS_DIR / "subscriber_type_comparison.png", dpi=150, bbox_inches="tight")
    plt.show()


### Insight 2: Streaming Activity by Subscriber Type

Do customers on different plans stream more or less? We'll use `num_streams` as our KPI.


if not df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(16, 7), sharey=True)
    fig.suptitle('Streaming Activity (num_streams) by Subscriber Type')
    
    plan_order = ['aavail_basic', 'aavail_premium', 'aavail_unlimited']
    
    # US Market
    sns.boxplot(data=df_us, x='subscriber_type', y='num_streams', ax=axes[0], order=plan_order)
    axes[0].set_title('United States')
    axes[0].set_xlabel('Subscriber Type')
    axes[0].set_ylabel('Number of Streams')
    
    # Singapore Market
    sns.boxplot(data=df_sg, x='subscriber_type', y='num_streams', ax=axes[1], order=plan_order)
    axes[1].set_title('Singapore')
    axes[1].set_xlabel('Subscriber Type')
    axes[1].set_ylabel('') # Remove y-label
    
    plt.tight_layout()
    plt.savefig(ASSETS_DIR / "streams_by_subscriber_type.png", dpi=150, bbox_inches="tight")
    plt.show()


### Insight 3: Subscriber Conversion (is_subscriber)

What is the ratio of subscribers to non-subscribers in each market?


In [ ]:
if not df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
    fig.suptitle('Subscriber Status (is_subscriber) by Market')
    
    # US Market
    us_counts = df_us['is_subscriber'].value_counts()
    sns.barplot(x=us_counts.index, y=us_counts.values, ax=axes[0], palette='pastel')
    axes[0].set_title('United States')
    axes[0].set_xlabel('Is Subscriber?')
    axes[0].set_ylabel('Count of Customers')
    axes[0].set_xticklabels(['True (Subscriber)', 'False (Non-Subscriber)'])
    
    # Singapore Market
    sg_counts = df_sg['is_subscriber'].value_counts()
    sns.barplot(x=sg_counts.index, y=sg_counts.values, ax=axes[1], palette='pastel')
    axes[1].set_title('Singapore')
    axes[1].set_xlabel('Is Subscriber?')
    axes[1].set_ylabel('') # Remove y-label
    axes[1].set_xticklabels(['True (Subscriber)', 'False (Non-Subscriber)'])
    
    plt.tight_layout()
    plt.savefig(ASSETS_DIR / "subscriber_status.png", dpi=150, bbox_inches="tight")
    plt.show()


### Insight 4: Age Demographics by Subscriber Type

Is there a difference in age for customers on various plans?


In [ ]:
if not df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(16, 7), sharey=True)
    fig.suptitle('Age Distribution by Subscriber Type')
    
    plan_order = ['aavail_basic', 'aavail_premium', 'aavail_unlimited']
    
    # US Market
    sns.boxplot(data=df_us, x='subscriber_type', y='age', ax=axes[0], order=plan_order)
    axes[0].set_title('United States')
    axes[0].set_xlabel('Subscriber Type')
    axes[0].set_ylabel('Age')
    
    # Singapore Market
    sns.boxplot(data=df_sg, x='subscriber_type', y='age', ax=axes[1], order=plan_order)
    axes[1].set_title('Singapore')
    axes[1].set_xlabel('Subscriber Type')
    axes[1].set_ylabel('') # Remove y-label
    
    plt.tight_layout()
    plt.savefig(ASSETS_DIR / "age_by_subscriber_type.png", dpi=150, bbox_inches="tight")
    plt.show()


## Step 6: Summary and Recommendations

Based on the cleaned and visualized data, here are the main takeaways for AAVAIL stakeholders.

**Key Takeaways:**

1.  **Imputed Data:** The `aavail_premium` plan was used to fill in missing `subscriber_type` data. This is a potential source of bias, as it was the most popular plan, but it was a necessary step to complete the dataset.

2.  **Plan Popularity:**
    * In both the **US and Singapore**, the `aavail_premium` plan is the most popular, followed by `aavail_basic`. The `aavail_unlimited` plan has a significantly smaller customer base in both markets.
    * The US market has a much larger customer base overall in this dataset.

3.  **Streaming Activity (`num_streams`):**
    * **United States:** There is no clear, significant difference in streaming activity between the three plans. The median number of streams is very similar across `basic`, `premium`, and `unlimited`. This is a critical finding: customers on the `unlimited` plan are not streaming significantly more than those on the `basic` plan.
    * **Singapore:** The pattern is similar to the US. The `aavail_unlimited` plan shows a slightly *lower* median stream count, and all three plans have a very similar distribution.
    * **Note:** The `aavail_unlimited` plan in Singapore also shows a very low outlier (near 0 streams), which may be a data entry error or a disengaged user.

4.  **Subscriber Conversion:**
    * Both markets show a very high conversion rate, with subscribers (`True`) significantly outnumbering non-subscribers (`False`). This suggests the platform is effective at converting users, or this dataset primarily consists of acquired customers.

5.  **Age Demographics:**
    * The age distributions are very similar across all subscriber types in both countries. The median age for all plans hovers around the mid-20s to 30.
    * There are some older outliers (up to 70s) in the Singapore market for `basic` and `premium` plans.

**Actionable Recommendations:**

1.  **Re-evaluate the `aavail_unlimited` Plan:** The data strongly suggests this plan is not providing its intended value (if the value is "unlimited streaming"). Customers are not using it more than other plans.
    * **Action:** Conduct user research (surveys, interviews) with `unlimited` subscribers to understand *why* they chose this plan. Is it for a specific feature (e.g., 4K, multiple screens) not captured in `num_streams`? Or do they *perceive* they will stream more, but don't?

2.  **Investigate Streaming Behavior:** Why is streaming activity so similar across all plans?
    * **Action:** The `num_streams` data has a wide, flat distribution. This could be monthly data, but it's unclear. The business needs to define what an "active" user is. Is 20 streams/month good? We need more context on this KPI.

3.  **Marketing & Pricing:**
    * Since `aavail_premium` is the most popular, marketing should continue to highlight its features.
    * **Action:** Consider a marketing campaign to upsell `basic` users to `premium`. Since `unlimited` is underperforming, either a) re-package it with more compelling, distinct features, or b) consider phasing it out and simplifying the offering to two tiers.

4.  **Data Quality:**
    * **Action:** Investigate the source of the missing `subscriber_type` data. Why are nearly 20% of customer records missing this? Fixing this data entry issue is crucial for future analysis.


In [ ]:
# Generate summary metrics for HTML report
if not df.empty:
    summary_metrics = (
        df_imputed.groupby("country_name").agg(
            users=("customer_id", "count"),
            avg_streams=("num_streams", "mean"),
            subscriber_pct=("is_subscriber", lambda s: (s.eq("True").mean() * 100) if s.dtype == 'object' else (s.mean() * 100)),
        )
        .round({"avg_streams": 2, "subscriber_pct": 1})
    )
    
    # Calculate additional metrics for the report
    subscriber_type_dist = df_imputed.groupby(['country_name', 'subscriber_type']).size().unstack(fill_value=0)
    
    print("Summary metrics calculated for HTML report generation.")


In [ ]:
## Step 7: Generate HTML Report

Export an HTML dashboard for business stakeholders with all visualizations and insights.


In [ ]:
if not df.empty:
    # Prepare data for HTML report
    us_subscriber_counts = df_us['subscriber_type'].value_counts().to_dict()
    sg_subscriber_counts = df_sg['subscriber_type'].value_counts().to_dict()
    
    # Calculate streaming stats by plan
    us_streams_by_plan = df_us.groupby('subscriber_type')['num_streams'].agg(['mean', 'median']).round(2)
    sg_streams_by_plan = df_sg.groupby('subscriber_type')['num_streams'].agg(['mean', 'median']).round(2)
    
    # Subscriber status counts
    us_sub_status = df_us['is_subscriber'].value_counts().to_dict()
    sg_sub_status = df_sg['is_subscriber'].value_counts().to_dict()
    
    dashboard_template = f"""<!DOCTYPE html>
<html lang='en'>
<head>
  <meta charset='UTF-8'>
  <meta name='viewport' content='width=device-width, initial-scale=1.0'>
  <title>AAVAIL Market Insights - US & Singapore</title>
  <script src='https://cdn.jsdelivr.net/npm/chart.js'></script>
  <link rel='preconnect' href='https://fonts.googleapis.com'>
  <link rel='preconnect' href='https://fonts.gstatic.com' crossorigin>
  <link href='https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap' rel='stylesheet'>
  <style>
    body {{ font-family: 'Inter', sans-serif; margin: 0; background: #f7f8fa; color: #1f2933; line-height: 1.6; }}
    header {{ background: linear-gradient(135deg, #2563eb, #60a5fa); color: white; padding: 2.5rem; }}
    header h1 {{ margin: 0 0 0.5rem 0; font-size: 2rem; }}
    header p {{ margin: 0; opacity: 0.95; }}
    main {{ padding: 2rem max(5vw, 2rem); max-width: 1400px; margin: 0 auto; }}
    .section {{ margin-bottom: 3rem; }}
    .section h2 {{ color: #1e40af; border-bottom: 2px solid #3b82f6; padding-bottom: 0.5rem; margin-bottom: 1.5rem; }}
    .grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(320px, 1fr)); gap: 2rem; margin-bottom: 2rem; }}
    .card {{ background: white; border-radius: 16px; padding: 1.5rem; box-shadow: 0 15px 35px rgba(15, 23, 42, 0.08); }}
    .card h3 {{ margin-top: 0; color: #1e40af; }}
    table {{ width: 100%; border-collapse: collapse; margin-top: 1rem; }}
    th, td {{ text-align: left; padding: 0.75rem 1rem; border-bottom: 1px solid #e2e8f0; }}
    th {{ background: #f8fafc; font-weight: 600; }}
    .insights {{ background: #eff6ff; border-left: 4px solid #2563eb; padding: 1.5rem; margin: 2rem 0; border-radius: 8px; }}
    .insights h3 {{ margin-top: 0; color: #1e40af; }}
    .insights ul {{ margin: 0.5rem 0; padding-left: 1.5rem; }}
    .insights li {{ margin: 0.5rem 0; }}
    .recommendations {{ background: #f0fdf4; border-left: 4px solid #10b981; padding: 1.5rem; margin: 2rem 0; border-radius: 8px; }}
    .recommendations h3 {{ margin-top: 0; color: #059669; }}
    img {{ max-width: 100%; height: auto; border-radius: 8px; margin: 1rem 0; }}
    .chart-container {{ position: relative; height: 300px; margin: 1rem 0; }}
  </style>
</head>
<body>
  <header>
    <h1>AAVAIL Streaming Market Insights</h1>
    <p>Comprehensive analysis of United States and Singapore markets after data imputation</p>
  </header>
  <main>
    <section class='section'>
      <h2>Market Overview</h2>
      <div class='grid'>
        <article class='card'>
          <h3>Market Summary</h3>
          <table>
            <thead>
              <tr><th>Market</th><th>Users</th><th>Avg Streams</th><th>Subscriber %</th></tr>
            </thead>
            <tbody>
              { ''.join(
                f"<tr><td>{country.replace('_', ' ').title()}</td><td>{row['users']:,}</td><td>{row['avg_streams']:.2f}</td><td>{row['subscriber_pct']:.1f}%</td></tr>"
                for country, row in summary_metrics.iterrows()
              ) }
            </tbody>
          </table>
        </article>
        <article class='card'>
          <h3>Average Streams by Market</h3>
          <div class='chart-container'>
            <canvas id='avgStreams'></canvas>
          </div>
        </article>
        <article class='card'>
          <h3>Subscriber Penetration</h3>
          <div class='chart-container'>
            <canvas id='subscriberPct'></canvas>
          </div>
        </article>
      </div>
    </section>

    <section class='section'>
      <h2>Subscriber Type Distribution</h2>
      <div class='grid'>
        <article class='card'>
          <h3>United States</h3>
          <div class='chart-container'>
            <canvas id='usSubscriberTypes'></canvas>
          </div>
        </article>
        <article class='card'>
          <h3>Singapore</h3>
          <div class='chart-container'>
            <canvas id='sgSubscriberTypes'></canvas>
          </div>
        </article>
      </div>
    </section>

    <section class='section'>
      <h2>Streaming Activity by Plan</h2>
      <div class='grid'>
        <article class='card'>
          <h3>US - Average Streams by Plan</h3>
          <table>
            <thead>
              <tr><th>Plan</th><th>Mean</th><th>Median</th></tr>
            </thead>
            <tbody>
              { ''.join(
                f"<tr><td>{plan.replace('aavail_', '').title()}</td><td>{row['mean']:.2f}</td><td>{row['median']:.2f}</td></tr>"
                for plan, row in us_streams_by_plan.iterrows()
              ) }
            </tbody>
          </table>
        </article>
        <article class='card'>
          <h3>Singapore - Average Streams by Plan</h3>
          <table>
            <thead>
              <tr><th>Plan</th><th>Mean</th><th>Median</th></tr>
            </thead>
            <tbody>
              { ''.join(
                f"<tr><td>{plan.replace('aavail_', '').title()}</td><td>{row['mean']:.2f}</td><td>{row['median']:.2f}</td></tr>"
                for plan, row in sg_streams_by_plan.iterrows()
              ) }
            </tbody>
          </table>
        </article>
      </div>
    </section>

    <section class='section'>
      <h2>Visualizations</h2>
      <div class='grid'>
        <article class='card'>
          <h3>Missing Data Heatmap</h3>
          <img src='assets/missingness_heatmap.png' alt='Missing Data Heatmap' />
        </article>
        <article class='card'>
          <h3>Subscriber Type Comparison</h3>
          <img src='assets/subscriber_type_comparison.png' alt='Subscriber Type Distribution' />
        </article>
        <article class='card'>
          <h3>Streaming Activity by Plan</h3>
          <img src='assets/streams_by_subscriber_type.png' alt='Streaming Activity by Subscriber Type' />
        </article>
        <article class='card'>
          <h3>Subscriber Status</h3>
          <img src='assets/subscriber_status.png' alt='Subscriber Status by Market' />
        </article>
        <article class='card'>
          <h3>Age Demographics</h3>
          <img src='assets/age_by_subscriber_type.png' alt='Age Distribution by Subscriber Type' />
        </article>
        <article class='card'>
          <h3>Imputation Comparison</h3>
          <img src='assets/imputation_comparison.png' alt='Before and After Imputation' />
        </article>
      </div>
    </section>

    <section class='section'>
      <div class='insights'>
        <h3>Key Insights</h3>
        <ul>
          <li><strong>Plan Popularity:</strong> The <code>aavail_premium</code> plan is the most popular in both US and Singapore markets, followed by <code>aavail_basic</code>. The <code>aavail_unlimited</code> plan has a significantly smaller customer base.</li>
          <li><strong>Streaming Activity:</strong> There is no clear, significant difference in streaming activity between the three plans. Customers on the <code>unlimited</code> plan are not streaming significantly more than those on the <code>basic</code> plan.</li>
          <li><strong>Subscriber Conversion:</strong> Both markets show a very high conversion rate, with subscribers significantly outnumbering non-subscribers.</li>
          <li><strong>Age Demographics:</strong> Age distributions are very similar across all subscriber types in both countries, with median age hovering around mid-20s to 30.</li>
        </ul>
      </div>

      <div class='recommendations'>
        <h3>Actionable Recommendations</h3>
        <ul>
          <li><strong>Re-evaluate the <code>aavail_unlimited</code> Plan:</strong> Conduct user research to understand why customers choose this plan if they're not streaming more than basic plan users.</li>
          <li><strong>Investigate Streaming Behavior:</strong> Define what an "active" user is and provide more context on the <code>num_streams</code> KPI.</li>
          <li><strong>Marketing & Pricing:</strong> Consider upselling <code>basic</code> users to <code>premium</code>. Evaluate whether to re-package or phase out the <code>unlimited</code> plan.</li>
          <li><strong>Data Quality:</strong> Investigate the source of missing <code>subscriber_type</code> data to prevent future data entry issues.</li>
        </ul>
      </div>
    </section>
  </main>
  <script>
    const labels = {list(summary_metrics.index)}.map(c => c.replace('_', ' ').replace(/\\b\\w/g, chr => chr.toUpperCase()));
    const avgStreams = {summary_metrics['avg_streams'].tolist()};
    const subscriberPct = {summary_metrics['subscriber_pct'].tolist()};

    const palette = ['#2563eb', '#10b981'];

    // Average Streams Chart
    new Chart(document.getElementById('avgStreams'), {{
      type: 'bar',
      data: {{ labels, datasets: [{{ label: 'Avg Streams', data: avgStreams, backgroundColor: palette }}] }},
      options: {{ 
        responsive: true, 
        maintainAspectRatio: false,
        plugins: {{ legend: {{ display: false }} }},
        scales: {{ y: {{ beginAtZero: true }} }}
      }}
    }});

    // Subscriber Penetration Chart
    new Chart(document.getElementById('subscriberPct'), {{
      type: 'bar',
      data: {{ labels, datasets: [{{ label: 'Subscriber %', data: subscriberPct, backgroundColor: palette }}] }},
      options: {{
        responsive: true,
        maintainAspectRatio: false,
        scales: {{ y: {{ beginAtZero: true, suggestedMax: 100, ticks: {{ callback: value => value + '%' }} }} }},
        plugins: {{ legend: {{ display: false }} }}
      }}
    }});

    // US Subscriber Types
    const usPlanLabels = {list(us_subscriber_counts.keys())}.map(p => p.replace('aavail_', '').replace('_', ' ').replace(/\\b\\w/g, chr => chr.toUpperCase()));
    const usPlanData = {list(us_subscriber_counts.values())};
    new Chart(document.getElementById('usSubscriberTypes'), {{
      type: 'doughnut',
      data: {{ 
        labels: usPlanLabels,
        datasets: [{{ data: usPlanData, backgroundColor: ['#3b82f6', '#10b981', '#f59e0b'] }}]
      }},
      options: {{ responsive: true, maintainAspectRatio: false }}
    }});

    // Singapore Subscriber Types
    const sgPlanLabels = {list(sg_subscriber_counts.keys())}.map(p => p.replace('aavail_', '').replace('_', ' ').replace(/\\b\\w/g, chr => chr.toUpperCase()));
    const sgPlanData = {list(sg_subscriber_counts.values())};
    new Chart(document.getElementById('sgSubscriberTypes'), {{
      type: 'doughnut',
      data: {{ 
        labels: sgPlanLabels,
        datasets: [{{ data: sgPlanData, backgroundColor: ['#3b82f6', '#10b981', '#f59e0b'] }}]
      }},
      options: {{ responsive: true, maintainAspectRatio: false }}
    }});
  </script>
</body>
</html>"""

    REPORT_PATH.write_text(dashboard_template, encoding="utf-8")
    print(f"HTML report generated at {REPORT_PATH}")


## 6. Next Steps

- Validate imputed values with domain experts and iterate if business rules evolve.
- Extend the analysis with cohort-specific churn / retention modeling.
- Automate CI checks to ensure notebooks conform to IBM’s reproducibility guidelines (IBM Developer Skills Network, 2024).


## Next Steps

- Validate imputed values with domain experts and iterate if business rules evolve.
- Extend the analysis with cohort-specific churn / retention modeling.
- Conduct user research on unlimited plan subscribers to understand their usage patterns.
- Investigate data quality issues causing missing subscriber_type values.
